**Input**

* `N` - ratio of Nitrogen (NH4+) content in soil
* `P` - ratio of Phosphorous (P) content in soil
* `K` - ratio of Potassium (K) content in soil
* `ph` - soil acidity (pH)
* `ec` - electrical conductivity
* `oc` - organic carbon
* `S` - sulfur (S)
* `zn` - Zinc (Zn)
* `fe` - Iron (Fe)
* `cu` - Copper (Cu)
* `Mn` - Manganese (Mn)
* `B` - Boron (B)

**Output**

* Class fertility (0 "Less Fertile", 1 "Fertile", 2 "Highly Fertile")

## Importation des Librairies et Modules

In [ ]:
!pip install scikit-learn

In [ ]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree, ensemble
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

## Chargement du jeu de données

In [ ]:
data = pd.read_csv('/content/dataset1.csv')
data.head()

## Exploration des données

In [ ]:
data.shape

In [ ]:
data.info()

Ce dataset à 13 colonnes composées comme suit:


*   12 colonnes representants les caractéristiques du sol.
*   La dernière colonne (`Output`) represente l'étiquette ie. si le sol est fertile



In [ ]:
data.describe()

In [ ]:
data['Output'].value_counts()

La colonne cible (`Output`) contient 3 classes:


*   `0` : représente un sol très peu fertile
*   `1` : représente un sol moyennement fertile
*   `2` : représente un sol très fertile



In [ ]:
corr = data.corr()

In [ ]:
f, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(corr,
            cmap=sns.color_palette('BuGn_r'),
            vmin=-1.0,
            vmax=1.0,
            square=True,
            ax=ax)

## Traitement des données

In [ ]:
# Renommons la colonne cible
data.rename(columns={'Output': 'fertility'}, inplace=True)
data.head()

In [ ]:
features = data.drop('fertility', axis=1)
label = data.loc[:, 'fertility']

In [ ]:
features.columns

In [ ]:
features.shape

In [ ]:
type(features)

In [ ]:
features.head()

In [ ]:
print(label.shape)

In [ ]:
label.head()

In [ ]:
features.hist(bins=50, figsize=(10, 10), color ='green', grid=False)
plt.show()

## Préparation des données pour former le modèle de Machine Learning

Nous utilisons la transformation logarithmique pour convertir une distribution asymétrique en une distribution normale.

In [ ]:
def transform_features(features):
    """
    Transforme les caractéristiques en prenant le logarithme base 10 si la caractéristique est numérique.

    Args:
    - features : DataFrame contenant les caractéristiques à transformer

    Returns:
    - transformed_features : DataFrame contenant les caractéristiques transformées
    """
    def log_transform(x):
        if np.issubdtype(x.dtype, np.number):
            return np.log10(x)
        else:
            return x

    transformed_features = features.apply(log_transform)
    return transformed_features

In [ ]:
featuresTransformed = transform_features(features)
featuresTransformed.hist(bins=50, figsize=(10, 10), color='green', grid=False)
plt.show()

### Séparation des données en données d'entrainement et de validation

In [ ]:
trainInput, validationInput, trainTarget, validationTarget = train_test_split(featuresTransformed, label, test_size = 0.2, shuffle=True, random_state = 42)

In [ ]:
trainInput.shape, validationInput.shape, trainTarget.shape, validationTarget.shape

## Processus du choix du meilleur Modèle

In [ ]:
trainTarget = trainTarget.values.ravel()

In [ ]:
svcClf = SVC()
svcClf.fit(trainInput, trainTarget)

In [ ]:
forestClf = ensemble.RandomForestClassifier()
forestClf.fit(trainInput, trainTarget)

In [ ]:
nbClf = GaussianNB()
nbClf.fit(trainInput, trainTarget)

In [ ]:
knnClf = KNeighborsClassifier()
knnClf.fit(trainInput, trainTarget)

In [ ]:
treeClf = tree.DecisionTreeClassifier()
treeClf.fit(trainInput, trainTarget)

In [ ]:
models = [svcClf, forestClf, nbClf, knnClf, treeClf]
accs = []
titles = []

for model in models:
    pred = model.predict((validationInput))
    model_acc = accuracy_score(validationTarget, pred)
    accs.append(model_acc)
    titles.append(type(model).__name__)
    print(type(model).__name__, " accuarcy is ", model_acc)

fig = plt.figure(figsize=(10, 5))
sns.barplot(x = titles, y=accs)
plt.xlabel("Modèles")  # Étiquette de l'axe x
plt.ylabel("Accuracies")


Nous constatons que le modèle RandomForest a la meilleure précision. Nous allons le maintenir et dans le suite nous allons apporter des modifications ses hyper-paramètres pour améliorer sa précision

## Fine Tuning de notre modèle RandForest

In [ ]:
forestClassifier = ensemble.RandomForestClassifier(random_state=42)

In [ ]:
paramGrid = {
    'n_estimators': [200, 300, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4, 5, 6 ,7 ,8, 9, 10],
    'criterion' :['gini', 'entropy']
}

In [ ]:
clf = GridSearchCV(
    estimator=forestClassifier,
    param_grid=paramGrid,
    cv= 5
  )
clf.fit(trainInput, trainTarget)

In [ ]:
clf.best_params_

## Entrainement du modèle

In [ ]:
randomForestModel = ensemble.RandomForestClassifier(criterion = 'gini',
 max_depth = 10,
 max_features = 'auto',
 n_estimators = 300, random_state=42)

In [ ]:
randomForestModel.fit(trainInput, trainTarget)

In [ ]:
predictions = randomForestModel.predict(validationInput)

In [ ]:
report = classification_report(validationTarget, predictions)
report

In [ ]:
mode_acc = accuracy_score(validationTarget, predictions)
mode_acc

#### Sauvegarde du modèle au format pkl

In [ ]:
# with open('random_forest_pkl.pkl', 'wb') as file:
#     pickle.dump(model, file)
import joblib

# Sauvegarder le modèle
joblib.dump(model, 'random_forest_pkl.pkl')

In [ ]:
import sklearn
print(sklearn.__version__)